# Topic Extraction with BERTopic

In [1]:
from bertopic import BERTopic
import re
from enum import Enum
from nltk.corpus import stopwords
from spacy.lang.de.stop_words import STOP_WORDS

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
class Party(Enum):
    AFD = 0
    CDU = 1
    FDP = 2
    GRUENE = 3
    LINKE = 4
    SPD = 5

Load party texts

In [ ]:
# stopwords
nltk_stopwords = stopwords.words('german')

# build stopwords list
all_stopwords = list(set(STOP_WORDS) | set(nltk_stopwords))
with open('custom_stopwords.txt', 'r', encoding='utf-8') as f:
    all_stopwords += [line.strip() for line in f.readlines()]

party_text = {}
for party in Party:
    all_stopwords.extend(['{}'.format(party.name.lower())])
    with open('resources/' + party.name + '.txt', encoding='utf-8', errors='ignore') as txt:
        file = " ".join(l for l in txt)
        # remove gender *
        file = re.sub(r'\*innen(\w*)\s', r'\1 ', file)
    party_text[party] = file

In [ ]:
def split_in_256_chunks(text):
    chunks = []
    tokens = text.split(' ')
    for i in range(0, len(tokens), 256):
        chunks.append(" ".join(tokens[i: min(i+255, len(tokens) - 1)]))
    return chunks

party_text = {p: split_in_256_chunks(t) for (p,t) in party_text.items()}

In [ ]:
model = BERTopic(language='german')
topics, probs = model.fit_transform(party_text[Party.FDP])

In [ ]:
model.visualize_topics()